In [3]:
%%writefile agentbe.py
from autogen_agentchat.agents import CodeExecutorAgent
import asyncio
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.base import TaskResult
from problemsolver import problem_solver_agent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor
import streamlit as st


async def main():
    docker=DockerCommandLineCodeExecutor(
        work_dir="/tmp",
        timeout=120
    )
    code_executor_agent=CodeExecutorAgent(
        name="CodeExecAgebnt",
        code_executor=docker
    )
    task=TextMessage(
        content="""Here is some code
        ```python
print('Hello world docker')```
        """, source="user"
    )

    #problem_solver_agent=problem_solver_agent
    termination=TextMentionTermination('TERMINATE')

    team=RoundRobinGroupChat([problem_solver_agent, code_executor_agent],
    termination_condition=termination,
    max_turns=10)


    
    
    await docker.start()
    try:

        task=""" Write a python code to calculate average of 10 random numbers"""
        async for message in team.run_stream(task=task):
            if isinstance(message, TextMessage):
                print('=='*20)
                print(message.source,':',message.content)
            elif isinstance(message, TaskResult):
                print("Stop Reason",message.stop_reason)
        result=await code_executor_agent.on_messages(
            messages=[task],
            cancellation_token=CancellationToken()
        )
        print(result.chat_message.content)
       
    except Exception as e:
        print(f"Error {e}")
    finally:
        print("Stopping Container")
        await docker.stop()
if __name__=="__main__":
    asyncio.run(main())

Overwriting agentbe.py


In [7]:
%%writefile agentbe_withfrontend.py
import asyncio
import streamlit as st

from autogen_agentchat.agents import CodeExecutorAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.base import TaskResult
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

from problemsolver import problem_solver_agent


def _run_async(coro):
    """
    Streamlit runs scripts top-to-bottom; this helper runs async code safely.
    """
    try:
        return asyncio.run(coro)
    except RuntimeError:
        # Fallback if an event loop is already running in the environment
        loop = asyncio.new_event_loop()
        try:
            return loop.run_until_complete(coro)
        finally:
            loop.close()


async def run_team_stream(problem: str, max_turns: int = 10):
    docker = DockerCommandLineCodeExecutor(work_dir="/tmp", timeout=120)
    code_executor_agent = CodeExecutorAgent(
        name="CodeExecAgent",
        code_executor=docker,
    )

    termination = TextMentionTermination("TERMINATE")

    team = RoundRobinGroupChat(
        [problem_solver_agent, code_executor_agent],
        termination_condition=termination,
        max_turns=max_turns,
    )

    await docker.start()
    try:
        async for message in team.run_stream(task=problem):
            if isinstance(message, TextMessage):
                yield f"**{message.source}:**\n\n{message.content}"
            elif isinstance(message, TaskResult):
                yield f"✅ **Stop reason:** {message.stop_reason}"
            else:
                # In case other message types appear (tool calls/results, etc.)
                src = getattr(message, "source", type(message).__name__)
                content = getattr(message, "content", repr(message))
                yield f"**{src}:**\n\n{content}"
    finally:
        await docker.stop()


def main():
    st.set_page_config(page_title="AlgoGenie", layout="wide")

    st.title("AlgoGenie — Agentic DSA Problem Solver [LOCAL]")
    st.write(
        "Ask a DSA problem. The problem-solver agent will generate a solution and the Docker executor will run code locally."
    )

    problem = st.text_area(
        "Enter your DSA problem",
        height=180,
        placeholder="Example: Write Python code to calculate the average of 10 random numbers.",
    )

    col1, col2 = st.columns([1, 3])
    with col1:
        run_btn = st.button("Run", type="primary")
    with col2:
        max_turns = st.slider("Max turns", min_value=2, max_value=30, value=10, step=1)

    st.divider()
    output = st.container()

    if run_btn:
        if not problem.strip():
            st.warning("Please enter a problem first.")
            return

        with output:
            st.subheader("Live output")
            placeholder = st.empty()
            log_lines = []

            async def _runner():
                async for line in run_team_stream(problem=problem.strip(), max_turns=max_turns):
                    log_lines.append(line)
                    # Render progressively
                    placeholder.markdown("\n\n---\n\n".join(log_lines))

            with st.spinner("Running..."):
                _run_async(_runner())


if __name__ == "__main__":
    main()


Overwriting agentbe_withfrontend.py


In [1]:
%%writefile .env
OPENAI_API_KEY="sk-proj-xGo9PS3L2KmEIi7uvgqOTkZtCAGYXzWw45IeKcIq45HiLuxtLdCVPT0Tdp3SSXsRqXW24GA" #keychanged

Overwriting .env


In [ ]:
%%writefile problemsolver.py
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
from autogen_agentchat.ui import Console
import os 
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent

load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")
model_client=OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key)
problem_solver_agent=AssistantAgent(
    name="DSASolverAgent",
    model_client=model_client,
    system_message="""   You are a overall problem solver agent, that is an expert in solving DSA problems in Python,
    Your working with your fellow code executor agent, 
    you can generate code outputs for the code executor agent to run,
    you write code to solve the task, your code will be in python, which the code executor agent can run,
    at the beginning of response, specify your plan to solve the task ,
    then you should give the code in a single code block that you can pass to the code executor agent.
    Once the code is executed, you have to make sure the output is correct.

    In the end once the code is executed successfully, you have to stay "TERMINATE"

      """                
)

Overwriting problemsolver.py


In [1]:
#%%writefile frontend.py
import streamlit as st
from agentbe import team
st.title("AlgoGenie- The Agentic DSA Problem Solver [LOCAL]")
st.write("Welcome to AlgoGenie, your personal DSA problem solver!, you can ask for code and run it locally!")

task=st.text_input("Enter your DSA problem")
if st.button("Run"):
    st.write("Running the task...")



ImportError: cannot import name 'team' from 'agentbe' (e:\Python DSA Master Agentic\agentbe.py)

In [6]:
%%writefile requirements.txt
autogen-agentchat
autogen-core
autogen-ext
asyncio
dotenv
openai
autogen-ext[docker]

Overwriting requirements.txt
